In [ ]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
import os

In [ ]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

num_classes = 10
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(x_train[4])

In [ ]:
from keras.applications import VGG16, ResNet50

# Generate a model with all layers (with top)
vgg16 = VGG16(weights='imagenet', 
              include_top=False, 
              input_shape=(128,128,3))

flatten = GlobalAveragePooling2D()(vgg16.layers[-1].output)

### 1. 當作 feature extractor 

In [ ]:
extractor = Model(input=vgg16.input, output=flatten)

In [ ]:
feature = extractor.predict(x_train[0:1])

### 2. 接上自己的 classifer 

In [ ]:
#Add a layer where input is the output of the  second last layer 
x = Dense(10, activation='softmax', name='predictions')(flatten)

#Then create the corresponding model 
model = Model(input=vgg16.input, output=x)
model.summary()

In [ ]:
# initiate Adam optimizer
opt = keras.optimizers.Adam(lr=0.0001, decay=1e-6)

# Let's train the model using adam
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs = 10

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)